In [20]:
import pandas as pd
import requests as r
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta, date
import numpy as np
import time
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:71.0) Gecko/20100101 Firefox/71.0'}

In [21]:
#https://stackoverflow.com/questions/23267409/how-to-implement-retry-mechanism-into-python-requests-library

import logging
import requests

from requests.adapters import HTTPAdapter, Retry

logging.basicConfig(level=logging.DEBUG)

s = requests.Session()
retries = Retry(total=5, backoff_factor=1, status_forcelist=[ 502, 503, 504 ])
s.mount('http://', HTTPAdapter(max_retries=retries))

In [22]:
trailing_num_days = 0

In [23]:
#get all apps > 10 m tvl
all_api = 'https://api.llama.fi/protocols'
res = pd.DataFrame( r.get(all_api, headers=header).json() )
res = res[res['tvl'] > 10_000_000] ##greater than 10mil
print(len(res))
# print(res.columns)


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.llama.fi:443
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocols HTTP/1.1" 200 1593379


356


In [24]:

protocols = res[['slug','chainTvls']]
re = res['chainTvls']
# r[1].keys()
protocols['chainTvls'] = protocols['chainTvls'].apply(lambda x: list(x.keys()) )
protocols

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_65434/1800157470.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,slug,chainTvls
0,makerdao,[Ethereum]
1,polygon-bridge-&-staking,[Polygon]
2,lido,"[Ethereum, Moonriver, Solana, Terra, Moonbeam]"
3,uniswap,"[Optimism, Ethereum, Polygon, Celo, Arbitrum]"
4,curve,"[Optimism, Harmony, Moonbeam, Ethereum, Aurora..."
...,...,...
351,guard(helmet),"[Binance, Polygon]"
352,aavegotchi,"[pool2, Polygon-pool2, Ethereum, Polygon, Poly..."
353,idex,"[Ethereum, Polygon]"
354,truefi,"[Ethereum, Ethereum-staking, Ethereum-borrowed..."


In [25]:
# protocols = protocols[ protocols['slug'] == 'uniswap-v3' ]
# api_str = 'https://api.llama.fi/protocol/'
# ad = pd.DataFrame( r.get(api_str).json()['chainTvls'] ).T[['tokens']]
# ad

In [26]:
# api_str = 'https://api.llama.fi/protocol/uniswap'
# prot_req = r.get(api_str, headers=header).json()['chainTvls']
# # prot_req['Ethereum']
# prot_req

In [27]:
#get by app
api_str = 'https://api.llama.fi/protocol/'
# print(protocols)
prod = []
for index,proto in protocols.iterrows():
#     print(proto)
    prot = proto['slug']
    chains = proto['chainTvls']
    apic = api_str + prot
#     time.sleep(0.1)
    try:
        prot_req = s.get(apic, headers=header).json()['chainTvls']
        print(apic)
        for ch in chains:
            ad = pd.json_normalize( prot_req[ch]['tokens'] )
            ad_usd = pd.json_normalize( prot_req[ch]['tokensInUsd'] )
#             ad = ad.merge(how='left')
            if not ad.empty:
                ad = pd.melt(ad,id_vars = ['date'])
                ad = ad.rename(columns={'variable':'token','value':'token_value'})
                ad_usd = pd.melt(ad_usd,id_vars = ['date'])
                ad_usd = ad_usd.rename(columns={'variable':'token','value':'usd_value'})
                ad = ad.merge(ad_usd,on=['date','token'])
                
                ad['date'] = pd.to_datetime(ad['date'], unit ='s') #convert to days
                ad['token'] = ad['token'].str.replace('tokens.','', regex=False)
                ad['protocol'] = prot
                ad['chain'] = ch
        #         ad['start_date'] = pd.to_datetime(prot[1])
                ad['date'] = ad['date'] - timedelta(days=1) #change to eod vs sod
                prod.append(ad)
    except:
        print('err')


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.llama.fi:443
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/makerdao HTTP/1.1" 200 1003980


https://api.llama.fi/protocol/makerdao


DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/polygon-bridge-&-staking HTTP/1.1" 307 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): defillama-datasets.s3.eu-central-1.amazonaws.com:443
DEBUG:urllib3.connectionpool:https://defillama-datasets.s3.eu-central-1.amazonaws.com:443 "GET /temp/protocol-Polygon%20Bridge%20&%20Staking.json HTTP/1.1" 200 14069111


https://api.llama.fi/protocol/polygon-bridge-&-staking


DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/lido HTTP/1.1" 200 443342


...

https://api.llama.fi/protocol/idex


DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/truefi HTTP/1.1" 200 588021


https://api.llama.fi/protocol/truefi


DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/gnosis-protocol-v1 HTTP/1.1" 200 2194555


https://api.llama.fi/protocol/gnosis-protocol-v1


In [28]:
df_df_all = pd.concat(prod)
df_df_all

,date,token,token_value,usd_value,protocol,chain
0,2022-05-09 00:00:00,MATIC,61211089.17387,5.060529e+07,makerdao,Ethereum
1,2022-05-10 00:00:00,MATIC,61211089.17387,5.437283e+07,makerdao,Ethereum
2,2022-05-11 00:00:00,MATIC,131231089.17387,8.719886e+07,makerdao,Ethereum
3,2022-05-12 00:00:00,MATIC,131264317.97387,7.932946e+07,makerdao,Ethereum
4,2022-05-13 00:00:00,MATIC,131264317.97387,9.167802e+07,makerdao,Ethereum
...,...,...,...,...,...,...
67579,2022-08-04 00:00:00,UNKNOWN (0xb6728900c317d41b8fdbd624879b02901f0...,NaN,NaN,gnosis-protocol-v1,Ethereum
67580,2022-08-05 00:00:00,UNKNOWN (0xb6728900c317d41b8fdbd624879b02901f0...,NaN,NaN,gnosis-protocol-v1,Ethereum
67581,2022-08-06 00:00:00,UNKNOWN (0xb6728900c317d41b8fdbd624879b02901f0...,NaN,NaN,gnosis-protocol-v1,Ethereum
67582,2022-08-07 00:00:00,UNKNOWN (0xb6728900c317d41b8fdbd624879b02901f0...,NaN,NaN,gnosis-protocol-v1,Ethereum


In [29]:
# df_df[df_df['protocol']=='perpetual-protocol']
#filter down a bit so we can do trailing comp w/o doing every row
df_df = df_df_all[df_df_all['date'].dt.date >= date.today()-timedelta(days=trailing_num_days +2) ]
display(df_df['date'].drop_duplicates())
#trailing comp
df_df['last_token_value'] = df_df.groupby(['token','protocol','chain'])['token_value'].shift(1)
#now actually filter
df_df = df_df[df_df['date'].dt.date >= date.today()-timedelta(days=trailing_num_days +1) ]
display(df_df['date'].drop_duplicates())
df_df[(df_df['protocol'] == 'uniswap') & (df_df['token'] != 'USDT') & (df_df['token_value'] > 0)]

89    2022-08-06 00:00:00
90    2022-08-07 00:00:00
91    2022-08-07 12:59:47
597   2022-08-07 12:59:44
23    2022-08-07 10:59:05
384   2022-08-07 09:59:08
40    2022-08-07 08:59:28
133   2022-08-07 13:15:15
468   2022-08-07 11:59:35
262   2022-08-07 10:58:53
Name: date, dtype: datetime64[ns]

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_65434/3806388026.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



90    2022-08-07 00:00:00
91    2022-08-07 12:59:47
597   2022-08-07 12:59:44
23    2022-08-07 10:59:05
384   2022-08-07 09:59:08
40    2022-08-07 08:59:28
133   2022-08-07 13:15:15
468   2022-08-07 11:59:35
262   2022-08-07 10:58:53
Name: date, dtype: datetime64[ns]

,date,token,token_value,usd_value,protocol,chain,last_token_value


In [30]:
# # DISTINCT TOKENS

# token_list = df_df.groupby(['token']).count()
# token_list = token_list[token_list['token_value']>=1000]
# token_list.sort_values(by='token_value',inplace=True, ascending=False)
# token_list.reset_index(inplace=True)
# missing_token_list = token_list[~token_list['token'].isin(cg_token_list)]
# missing_token_list
# missing_token_list

In [31]:
# data_df = df_df.merge(cg_df, on=['date','token'],how='inner')
data_df = df_df.copy()
data_df['token_value'] = data_df['token_value'].replace(0, np.nan)
data_df['price_usd'] = data_df['usd_value']/data_df['token_value']

# print(len(data_df))
data_df.sort_values(by='date',inplace=True)
# print(len(data_df))
# data_df['net_token_flow'] = data_df.groupby(['token','protocol','chain'])['token_value'].apply(lambda x: x- x.shift(1))
data_df['net_token_flow'] = data_df['token_value'] - data_df['last_token_value']

# print(len(data_df))
data_df['net_dollar_flow'] = data_df['net_token_flow'] * data_df['price_usd']


# data_df = data_df[data_df['token'] != 'RVRS'] #Harmony error
# data_df = data_df[data_df['token'] != 'usn'] #Harmony error
# data_df = data_df[data_df['protocol'] != 'shibaswap'] #Harmony error
data_df = data_df[abs(data_df['net_dollar_flow']) < 50_000_000_000] #50 bil error bar
data_df = data_df[~data_df['net_dollar_flow'].isna()] #50 bil error bar
# display(data_df[
# #         (data_df['protocol'] == 'sushiswap') & (data_df['chain'] == 'Harmony')
#         (data_df['net_dollar_flow'].notna()) & (data_df['net_dollar_flow'] != 0)
#         ].sort_values('net_dollar_flow'))


data_df.sort_values('net_dollar_flow')


,date,token,token_value,usd_value,protocol,chain,last_token_value,price_usd,net_token_flow,net_dollar_flow
3094,2022-08-07 12:59:47,WETH,4.126229e+05,7.460923e+08,aave-v2,Ethereum,463883.84492,1808.170,-51260.98422,-92688573.836962
809,2022-08-07 12:59:47,USDC,7.897567e+08,7.905465e+08,arrakis-finance,Ethereum,817700412.60739,1.001,-27943715.10742,-27971658.822528
404,2022-08-07 12:59:47,DAI,9.250450e+08,9.259701e+08,arrakis-finance,Ethereum,952785621.5319,1.001,-27740603.29238,-27768343.895672
595,2022-08-07 12:59:44,USDC,1.689407e+08,1.691097e+08,aave-v3,Optimism,193634769.65423,1.001,-24694042.67894,-24718736.72162
31826,2022-08-07 00:00:00,WETH,2.827280e+05,4.825686e+08,curve,Ethereum,292888.91286,1706.830,-10160.91489,-17342954.36183
...,...,...,...,...,...,...,...,...,...,...
3094,2022-08-07 12:59:47,WETH,4.128449e+05,7.464938e+08,aave-v2,Ethereum-borrowed,359581.72313,1808.170,53263.17428,96308873.83842
744,2022-08-07 12:59:44,USDC,9.173887e+08,9.183061e+08,aave-v3,Optimism-borrowed,809319900.479,1.001,108068841.13133,108176909.972461
2681,2022-08-07 12:59:44,USDC,1.654799e+09,1.656454e+09,aave-v3,borrowed,1545163401.46484,1.001,109635590.35699,109745225.947348
19807,2022-08-07 12:59:47,STETH,9.240958e+05,1.609137e+09,aave-v2,Ethereum,839423.32643,1741.310,84672.48895,147441051.734105


In [32]:
data_df[['date']].drop_duplicates().sort_values('date')
# data_df[data_df['date'].dt.date != data_df['date']].sort_values('date')

,date
90,2022-08-07 00:00:00
409,2022-08-07 08:59:28
384,2022-08-07 09:59:08
3944,2022-08-07 10:58:53
239,2022-08-07 10:59:05
468,2022-08-07 11:59:35
471941,2022-08-07 12:59:44
839,2022-08-07 12:59:47
401,2022-08-07 13:15:15


In [33]:
# data_df[data_df['protocol']=='perpetual-protocol'].sort_values(by='date')

# segment_cols = ['date','protocol','chain','token']

In [34]:
netdf_df = data_df[['date','protocol','chain','net_dollar_flow','usd_value']]
netdf_df = netdf_df.fillna(0)
netdf_df = netdf_df.groupby(['date','protocol','chain']).sum(['net_dollar_flow','usd_value']) ##agg by app
# display(netdf_df)
#usd_value is the TVL on a given day
netdf_df = netdf_df.groupby(['date','protocol','chain','usd_value']).sum(['net_dollar_flow'])
netdf_df.reset_index(inplace=True)
# display(netdf_df)
netdf_df['cumul_net_dollar_flow'] = netdf_df[['protocol','chain','net_dollar_flow']]\
                                    .groupby(['protocol','chain']).cumsum()
netdf_df.reset_index(inplace=True)
netdf_df.drop(columns=['index'],inplace=True)


In [35]:
#get latest
netdf_df['rank_desc'] = netdf_df.groupby(['protocol', 'chain'])['date'].\
                            rank(method='dense',ascending=False).astype(int)
# netdf_df[netdf_df['chain'] == 'Klaytn']

In [36]:
summary_df = netdf_df[( netdf_df['rank_desc'] == 1 ) &\
                        (~netdf_df['chain'].str.contains('borrowed')) &\
                        (~netdf_df['chain'].str.contains('staking'))  
                        ]
summary_df = summary_df.sort_values(by='cumul_net_dollar_flow',ascending=False)
summary_df['pct_of_tvl'] = 100* summary_df['net_dollar_flow'] / summary_df['usd_value']
summary_df['flow_direction'] = np.where(summary_df['cumul_net_dollar_flow']>=0,1,-1)
summary_df['abs_cumul_net_dollar_flow'] = abs(summary_df['cumul_net_dollar_flow'])
# display(summary_df)
fig = px.treemap(summary_df[summary_df['abs_cumul_net_dollar_flow'] !=0], \
                 path=[px.Constant("all"), 'chain', 'protocol'], \
#                  path=[px.Constant("all"), 'token', 'chain', 'protocol'], \
                 values='abs_cumul_net_dollar_flow', color='flow_direction'
#                 ,color_discrete_map={'-1':'red', '1':'green'})
                ,color_continuous_scale='Spectral'
                )
# fig.data[0].textinfo = 'label+text+value'
fig.update_traces(root_color="lightgrey")
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
# fig.update_layout(tickprefix = '$')
fig.show()

/Users/michaelsilberling/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:1616: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/michaelsilberling/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:1616: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/michaelsilberling/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:1616: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [37]:
# netdf_df_plot = netdf_df[netdf_df['chain']=='Ethereum']

# fig = px.line(netdf_df_plot, x="date", y="net_dollar_flow", color="protocol", \
#              title="Daily Net Dollar Flow since Program Announcement",\
#             labels={
#                      "date": "Day",
#                      "net_dollar_flow": "Net Dollar Flow (N$F)"
#                  }
#             )
# fig.update_layout(
#     legend_title="App Name"
# )
# fig.update_layout(yaxis_tickprefix = '$')
# fig.show()

# # cumul_fig = px.area(netdf_df, x="date", y="cumul_net_dollar_flow", color="protocol", \
# #              title="Cumulative Dollar Flow since Program Announcement",\
# #                    labels={
# #                      "date": "Day",
# #                      "cumul_net_dollar_flow": "Cumulative Net Dollar Flow (N$F)"
# #                  }
# #             ,areamode='group')
# # cumul_fig.update_layout(yaxis_tickprefix = '$')
# # cumul_fig.show()


# cumul_fig = go.Figure()
# proto_names = netdf_df_plot['protocol'].drop_duplicates()
# print(proto_names)
# for p in proto_names:
#     cumul_fig.add_trace(go.Scatter(x=netdf_df_plot[netdf_df_plot['protocol'] == p]['date'] \
#                                    , y=netdf_df_plot[netdf_df_plot['protocol'] == p]['cumul_net_dollar_flow'] \
#                                     ,name = p\
#                                   ,fill='tozeroy')) # fill down to xaxis

# cumul_fig.update_layout(yaxis_tickprefix = '$')
# cumul_fig.update_layout(
#     title="Cumulative Dollar Flow since Program Announcement",
#     xaxis_title="Day",
#     yaxis_title="Cumulative Net Dollar Flow (N$F)",
#     legend_title="App Name",
# #     color_discrete_map=px.colors.qualitative.G10
# )
# cumul_fig.show()